# Neurodocker

_Neurodocker_ is a Python project that generates custom Dockerfiles for neuroimaging and minifies existing Docker images (using [ReproZip](https://www.reprozip.org/)). The package can be used from the command-line or within a Python script. The command-line interface generates Dockerfiles and minifies Docker images, but interaction with the Docker Engine is left to the various `docker` commands. Within a Python script, however, _Neurodocker_ can generate Dockerfiles, build Docker images, run commands within resulting containers (using the [`docker` Python package](https://github.com/docker/docker-py)), and minify Docker images. The project is used for regression testing of [Nipype](https://github.com/nipy/nipype/) interfaces.

Examples in this notebook:
  - Command-line
    - [Generate Dockerfile](#generate-dockerfile)
    - [Generate Dockerfile (full)](#generate-dockerfile-full)
  - In a Python script
    - [Generate Dockerfile, build Docker image, run commands in image (minimal)](#generate-dockerfile-build-docker-image-run-commands-in-image-minimal)
    - [Generate full Dockerfile](#generate-full-dockerfile)
      - [Generated Dockerfile](examples/generated-full.Dockerfile)
  - Minimize Docker image
    - [Minimize existing Docker image](#minimize-existing-docker-image)
    - [Example of minimizing Docker image for FreeSurfer recon-all](https://github.com/freesurfer/freesurfer/issues/70#issuecomment-316361886)


## Note to users

This software is still in the early stages of development. If you come across an issue or a way to improve _Neurodocker_, please submit an issue or a pull request.

# Installation

You can install _Neurodocker_ with `pip`, or you can use the project's Docker image.

`pip install https://github.com/kaczmarj/neurodocker/archive/master.tar.gz`

or

`docker run --rm kaczmarj/neurodocker --help`

Note that building and minifying Docker images is not possible within the _Neurodocker_ Docker image.

# Supported Software

Valid options for each software package are the keyword arguments for the class that installs that package. These classes live in [`neurodocker.interfaces`](neurodocker/interfaces/). The default installation behavior for every software package (except Miniconda) is to install by downloading and un-compressing the binaries.


| software | argument | description |
| -------- | -------- | ----------- |
| **AFNI** | version* | Either 17.2.02 or latest. |
| **ANTs** | version* | 2.2.0, 2.1.0, 2.0.3, or 2.0.0 |
|          | use_binaries | If true (default), use pre-compiled binaries. If false, build from source. |
|          | git_hash  | Git hash to checkout to before building from source (only used if use_binaries is false). |
| **FreeSurfer** | version* | Any version for which binaries are provided. |
|                | license_path | Relative path to license file. If provided, this file will be copied into the Docker image. Must be within the build context. |
|                | min | If true, install a version of FreeSurfer minimized for recon-all. See [freesurfer/freesurfer#70](https://github.com/freesurfer/freesurfer/issues/70). False by default. |
| **FSL**** | version* | Any version for which binaries are provided. |
|           | use_binaries | If true (default), use pre-compiled binaries. Building from source is not available now but might be added in the future. |
|           | use_installer | If true, use FSL's Python installer. Only valid on CentOS images. |
| **Miniconda** | env_name* | Name of this conda environment. |
|               | python_version* | Version of Python. |
|               | conda_install | Packages to install with conda. e.g., `conda_install="numpy traits"` |
|               | pip_install | Packages to install with pip. |
|               | conda_opts  | Command-line options to pass to [`conda create`](https://conda.io/docs/commands/conda-create.html). e.g., `conda_opts="-c vida-nyu"` |
|               | pip_opts    | Command-line options to pass to [`pip install`](https://pip.pypa.io/en/stable/reference/pip_install/#options). |
|               | add_to_path | If true (default), add this environment to $PATH. |
|               | miniconda_version | Version of Miniconda. Latest by default. |
| **MRtrix3** | use_binaries | If true (default), use pre-compiled binaries. If false, build from source. |
|             | git_hash | Git hash to checkout to before building from source (only used if use_binaries is false). |
| **NeuroDebian** | os_codename* | Codename of the operating system (e.g., stretch, zesty). |
|                 | download_server* | Server to download NeuroDebian packages from. Choose the one closest to you. See `neurodocker generate --help` for the full list of servers. |
|                 | pkgs | Packages to download from NeuroDebian. |
|                 | full | If true (default), use non-free sources. If false, use libre sources. |
| **SPM** | version        | 12 (earlier versions will be supported in the future). |
|         | matlab_version | R2017a (other MCR versions will be supported once earlier SPM versions are supported). |


\* required argument.

** FSL is non-free. If you are considering commercial use of FSL, please consult the [relevant license](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/Licence).

# Examples

## Generate Dockerfile

Generate Dockerfile, and print result to stdout. The result can be piped to `docker build` to build the Docker image.

```shell
docker run --rm kaczmarj/neurodocker generate -b ubuntu:17.04 -p apt --ants version=2.2.0

docker run --rm kaczmarj/neurodocker generate -b ubuntu:17.04 -p apt --ants version=2.2.0 | docker build -
```

## Generate Dockerfile (full)

In this example, a Dockerfile is generated with all of the software that _Neurodocker_ supports, and the Dockerfile is saved to disk. The order in which the arguments are given is preserved in the Dockerfile. The saved Dockerfile can be passed to `docker build`.

```shell
# Generate Dockerfile.
docker run --rm kaczmarj/neurodocker generate \
--base debian:stretch --pkg-manager apt \
--install git vim \
--afni version=latest \
--ants version=2.2.0 \
--freesurfer version=6.0.0 min=true \
--fsl version=5.0.10 \
--user=neuro \
--miniconda env_name=default \
            python_version=3.5.1 \
            conda_opts="--channel vida-nyu" \
            conda_install="numpy pandas reprozip traits" \
            pip_install="nipype" \
--miniconda env_name=py27 \
            python_version=2.7 \
            add_to_path=false \
--user=root \
--mrtrix3 \
--neurodebian os_codename="jessie" \
              download_server="usa-nh" \
              pkgs="dcm2niix git-annex-standalone" \
--spm version=12 matlab_version=R2017a \
--user=neuro \
--env KEY_A=VAL_A KEY_B=VAL_B \
--env KEY_C="based on \$KEY_A" \
--instruction='RUN mkdir /opt/mydir' \
--add-to-entrypoint 'echo hello world' 'source myfile.sh' \
--expose 8888 \
--workdir /home/neuro \
--no-check-urls > examples/generated-full.Dockerfile

# Build Docker image using the saved Dockerfile.
docker build -t myimage -f generated-full.Dockerfile examples
```

Here is the [Dockerfile](examples/generated-full.Dockerfile) generated by the command above.

## Exercise 1: 

Create a docker image based on `neurodebin:stretch-non-free` and use apt to install `FSL`. Determine the FSL version.
Then create another image based on the same base image but with FSL `5.0.10`.

```bash
docker run --rm kaczmarj/neurodocker generate \
--base neurodebian:stretch-non-free --pkg-manager apt \
--install fsl-complete | docker build -t fsl-unknown -


docker run -it --rm fsl-unknown cat /usr/share/fsl/etc/fslversion
```

```bash

docker run --rm kaczmarj/neurodocker generate \
--base neurodebian:stretch-non-free --pkg-manager apt \
--fsl version=5.0.10 | docker build -t fsl5.10 -
```

## Generate Dockerfile, build Docker image, run commands in image (minimal)

In this example, a dictionary of specifications is used to generate a Dockerfile. A Docker image is built from the string representation of the Dockerfile. A container is started from that container, and commands are run within the running container. When finished, the container is stopped and removed.


```python
from neurodocker import Dockerfile
from neurodocker.docker import DockerImage, DockerContainer

specs = {
    'pkg_manager': 'apt',
    'check_urls': False,
    'instructions': [
        ('base', 'ubuntu:17.04'),
        ('ants', {'version': '2.2.0'})
    ]
}
# Create Dockerfile.
df = Dockerfile(specs)

# Build image.
image = DockerImage(df).build(log_console=False, log_filepath="build.log")

# Start container, and run commands.
container = DockerContainer(image).start()
container.exec_run('antsRegistration --help')
container.exec_run('ls /')
container.cleanup(remove=True)
```

## Generate full Dockerfile

In this example, we create a Dockerfile with all of the software that _Neurodocker_ supports, and we supply arbitrary Dockerfile instructions.

```python
from neurodocker import Dockerfile

specs = {
    'pkg_manager': 'apt',
    'check_urls': False,
    'instructions': [
        ('base', 'ubuntu:17.04'),
        ('install', ['git', 'vim']),
        ('user', 'neuro'),
        ('miniconda', {
            'env_name': 'my_env',
            'python_version': '3.5.1',
            'conda_install': 'traits',
            'pip_install': 'https://github.com/nipy/nipype/archive/master.tar.gz'}),
        ('user', 'root'),
        ('afni', {'version': 'latest'}),
        ('ants', {'version': '2.2.0'}),
        ('freesurfer', {'version': '6.0.0', 'license_path': 'rel/path/license.txt'}),
        ('fsl', {'version': '5.0.10', 'use_binaries': True}),
        ('mrtrix3', {'use_binaries': False}),
        ('neurodebian', {'os_codename': 'zesty', 'download_server': 'usa-nh',
                         'pkgs': ['afni', 'dcm2niix']}),
        ('spm', {'version': '12', 'matlab_version': 'R2017a'}),
        ('instruction', 'RUN echo "Hello, World"'),
        ('copy', ['rel/path/to/startup.sh', '/path/in/container/']),
        ('user', 'neuro'),
        ('env', {'KEY_A': 'VAL_A', 'KEY_B': 'VAL_B is "hello"'}),
        ('env', {'KEY_C': 'based on $KEY_A'}),
    ]
}

df = Dockerfile(specs)
df.save('path/to/Dockerfile')
print(df)
```

## Using reprozip to capture some analysis

A walkthrough is available here: http://nipy.org/workshops/2017-03-boston/lectures/repeatable-research/

In [1]:
%%bash

reprozip trace -d tracedir nib-ls /data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz

/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz float32 [256, 156, 256] 1.00x1.30x1.00   sform

Configuration file written in tracedir/config.yml
Edit that file then run the packer -- use 'reprozip pack -h' for help



Uploading usage statistics is currently disabled
Please help us by providing anonymous usage statistics; you can enable this
by running:
    reprozip usage_report --enable
If you do not want to see this message again, you can run:
    reprozip usage_report --disable
Nothing will be uploaded before you opt in.


## View the trace

In [2]:
%%bash

cat tracedir/config.yml

# ReproZip configuration file
# This file was generated by reprozip 1.0.9 at 2017-08-11T22:12:35.088333

# You might want to edit this file before running the packer
# See 'reprozip pack -h' for help

# Run info
version: "0.8"
runs:
# Run 0
- architecture: x86_64
  argv: [nib-ls, /data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz]
  binary: /opt/conda/envs/neuro/bin/nib-ls
  distribution: [debian, '9.1']
  environ: {CLICOLOR: '1', CONDA_DIR: /opt/conda, GIT_PAGER: cat, HOME: /home/neuro,
    HOSTNAME: 10eab4a5b4ea, JPY_PARENT_PID: '11', LANG: C.UTF-8, LC_ALL: C, LS_COLORS: 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.Z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:

## Pack the necessary files

In [3]:
%%bash

reprozip pack -d tracedir nibls.rpz


Uploading usage statistics is currently disabled
Please help us by providing anonymous usage statistics; you can enable this
by running:
    reprozip usage_report --enable
If you do not want to see this message again, you can run:
    reprozip usage_report --disable
Nothing will be uploaded before you opt in.


## Unpack the trace on a different container

In [4]:
%%bash 

reprounzip chroot setup --dont-bind-magic-dirs nibls.rpz nibbler

[REPROUNZIP] 22:14:21.013 WARNING: Not running as root, won't restore files' owner/group

Uploading usage statistics is currently disabled
Please help us by providing anonymous usage statistics; you can enable this
by running:
    reprounzip usage_report --enable
If you do not want to see this message again, you can run:
    reprounzip usage_report --disable
Nothing will be uploaded before you opt in.


In [5]:
%%bash

tree -L 2 nibbler

nibbler
|-- config.yml
|-- inputs.tar.gz
`-- root
    |-- bin -> usr/bin
    |-- data
    |-- etc
    |-- home
    |-- lib -> usr/lib
    |-- lib64 -> usr/lib64
    |-- opt
    `-- usr

9 directories, 2 files


## Rerun the analysis

In [1]:
%%bash

reprounzip chroot run nibbler

chroot: cannot change root directory to 'nibbler/root': Operation not permitted

*** Command finished, status: 125

Uploading usage statistics is currently disabled
Please help us by providing anonymous usage statistics; you can enable this
by running:
    reprounzip usage_report --enable
If you do not want to see this message again, you can run:
    reprounzip usage_report --disable
Nothing will be uploaded before you opt in.


## Minimize existing Docker image

In the following example, a Docker image is built with ANTs version 2.2.0 and a functional scan. The image is minified for running `antsMotionCorr`. The original ANTs Docker image is 1.85 GB, and the "minified" image is 365 MB.

```shell
# Create a Docker image with ANTs, and download a functional scan.
download_cmd="RUN curl -sSL -o /home/func.nii.gz http://psydata.ovgu.de/studyforrest/phase2/sub-01/ses-movie/func/sub-01_ses-movie_task-movie_run-1_bold.nii.gz"
neurodocker generate -b centos:7 -p yum --ants version=2.2.0 --instruction="$download_cmd" | docker build -t ants:2.2.0 -

# Run the container.
docker run --rm -it --name ants-reprozip-container --security-opt=seccomp:unconfined ants:2.2.0

# (in a new terminal window)
# Output a ReproZip pack file in ~/neurodocker-reprozip-output with the files
# necessary to run antsMotionCorr.
# See https://github.com/stnava/ANTs/blob/master/Scripts/antsMotionCorrExample
cmd="antsMotionCorr -d 3 -a /home/func.nii.gz -o /home/func_avg.nii.gz"
neurodocker reprozip ants-reprozip-container "$cmd"

reprozip docker setup neurodocker-reprozip.rpz test
```